In [1]:
from text_preprocessor import CorpusToEmbedding

import pandas as pd

df = pd.read_csv("./preprocessed/train.csv")
df_test = pd.read_csv("./preprocessed/test_seen.csv")

import pickle

with open("preprocessed/text_object", 'rb') as f:
    text_obj = pickle.load(f)
    f.close()

with open("preprocessed/text_object_test_seen", 'rb') as f:
    text_obj_test = pickle.load(f)
    f.close()

In [2]:
df["text_tokens"] = text_obj.tokens
df["labels_tokens"] = text_obj.labels
df_test["labels_tokens"] = text_obj_test.labels
df_test["text_tokens"] = text_obj_test.tokens

In [3]:
df_test

,Unnamed: 0,id,img,label,text,labels,labels_tokens,text_tokens
0,0,16395,img/16395.png,1,handjobs sold seperately,['person'],"[361, 999999, 999999]","[675754, 1178, 57402, 999999, 999999, 999999, ..."
1,1,37405,img/37405.png,1,introducing fidget spinner for women,['person'],"[361, 999999, 999999]","[5462, 150335, 30487, 14, 992, 999999, 999999,..."
2,2,94180,img/94180.png,1,happy pride month let's go beat up lesbians,"['person', 'handbag']","[361, 30160, 999999]","[1695, 5933, 875, 659, 2004, 321, 2268, 89, 98..."
3,3,54321,img/54321.png,1,laughs in [majority of u.s crime rate],"['person', 'person', 'person']","[361, 361, 361]","[6379, 6, 1534, 4, 30, 1405, 950, 999999, 9999..."
4,4,97015,img/97015.png,1,finds out those 72 virgins.. are goats,"['person', 'person', 'person', 'person', 'pers...","[361, 361, 361]","[300, 94, 128, 5192, 19708, 32, 14685, 999999,..."
...,...,...,...,...,...,...,...,...
995,995,3869,img/03869.png,0,a mother's love for the child is a divine thing,"['person', 'person', 'cat']","[361, 361, 3511]","[1043, 650, 14, 1, 815, 13, 7473, 383, 999999,..."
996,996,23817,img/23817.png,0,sea monkeys,"['bird', 'teddy_bear']","[4356, 3325, 999999]","[1976, 11922, 999999, 999999, 999999, 999999, ..."
997,997,56280,img/56280.png,0,little miss muffet sat on her tuffet,"['person', 'tie', 'chair', 'couch']","[361, 4540, 3408]","[452, 4196, 6731, 15, 84, 750018, 999999, 9999..."
998,998,29384,img/29384.png,0,they're in a row,"['parking_meter', 'parking_meter', 'toilet', '...","[9113, 9113, 8189]","[70, 32, 6, 4031, 999999, 999999, 999999, 9999..."


In [4]:
from torchvision import datasets, transforms
import torch

/home/vincent/miniforge3/envs/hateful-memes-classification/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/vincent/miniforge3/envs/hateful-memes-classification/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [5]:
import numpy as np

df["new_labels"] = np.where(df.label == 0, 1, 0)
df["new_id"] = df.id.astype(str) # for lexsorting
df_test["new_labels"] = np.where(df_test.label == 0, 1, 0)
df_test["new_id"] = df_test.id.astype(str) # for lexsorting

In [6]:
df = df.sort_values(["new_labels", "new_id"])
df_test = df_test.sort_values(["new_labels", "new_id"])

In [7]:
texts = torch.utils.data.TensorDataset(torch.tensor(df.text_tokens), torch.tensor(df.new_labels))

labels = torch.utils.data.TensorDataset(torch.tensor(df.labels_tokens), torch.tensor(df.new_labels))

texts_test = torch.utils.data.TensorDataset(torch.tensor(df_test.text_tokens), torch.tensor(df_test.new_labels))

labels_test = torch.utils.data.TensorDataset(torch.tensor(df_test.labels_tokens), torch.tensor(df_test.new_labels))

In [8]:
transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

images = datasets.ImageFolder("./preprocessed/img", transform=transformations)
images_test = datasets.ImageFolder("./preprocessed/test_seen", transform=transformations)

In [9]:
from torch.utils.data.sampler import RandomSampler

random_sampler = RandomSampler(images)
random_sampler_test = RandomSampler(images_test)

In [10]:
image_loader = torch.utils.data.DataLoader(
    images, batch_size=8, sampler=random_sampler
)
text_loader = torch.utils.data.DataLoader(texts, batch_size=8, sampler=random_sampler)

labels_loader = torch.utils.data.DataLoader(labels, batch_size=8, sampler=random_sampler)

image_loader_test = torch.utils.data.DataLoader(
    images_test, batch_size=8, sampler=random_sampler_test
)
text_loader_test = torch.utils.data.DataLoader(texts_test, batch_size=8, sampler=random_sampler_test)

labels_loader_test = torch.utils.data.DataLoader(labels_test, batch_size=8, sampler=random_sampler_test)

In [11]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        drp = 0.1
        n_classes = 2

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(text_obj.weights))
        self.embedding.requires_grad = False

        self.lstm = nn.LSTM(300, 30, bidirectional=False, batch_first=True)
        self.text_linear = nn.Sequential(
            nn.Linear(30, 15),
            nn.Linear(15, n_classes)
        )

        self.cnn_layer1 = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(drp),
            # Layer 2
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(drp),
            # # Layer 3
            # nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=0),
            # # Out: 62, 29
            # nn.BatchNorm2d(32),
            # nn.ReLU(inplace=True),
            # nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
            # # Out: 30, 14
            # nn.Dropout(drp),
        )

        self.fc1 = nn.Linear(224 * 224 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, n_classes)

    def forward(self, x):
        # x = torch.unsqueeze(x, dim=1)
        x = x[1]
        h_embedding = self.embedding(x)
        h_lstm, _ = self.lstm(h_embedding)
        out = self.text_linear(h_lstm)
        # out1 = self.cnn_layer1(x)
        # out2 = self.cnn_layer2(x)
        # full_out = torch.cat([out1, out2], dim=1)
        # full_out = full_out.view(full_out.size(0), -1)
        # x = torch.flatten(out1, start_dim=1)
        # x = self.dropout(self.relu(self.fc1(x)))
        # x = self.dropout(self.relu(self.fc2(x)))
        # x = self.dropout(self.relu(self.fc3(x)))
        # x = self.dropout(self.relu(self.fc4(x)))
        # x = self.dropout(self.relu(self.fc5(x)))
        # out = self.out(x)
        return out

In [12]:
from torchvision import datasets, models, transforms
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_conv = models.resnet152(pretrained=True)
# for param in model_conv.parameters():
#     param.requires_grad = False
model_conv.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
class Model(nn.Module):
    def __init__(self, weights) -> None:
        super().__init__()

        self.drp = 0.3
        self.n_classes = 2

        # self.ReLU = nn.ReLU()
        self.dropout = nn.Dropout(self.drp)
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights))

        # Text LSTM
        self.lstm = nn.LSTM(
            300, 30, bidirectional=False, batch_first=True, num_layers=3, dropout=self.drp
        )

        # LSTM linear
        self.lstm_linear = nn.Sequential(
            nn.Linear(900, 450),
            nn.ReLU(inplace=True),
            nn.Linear(450, 200),
            nn.Sigmoid(),
        )

        # Text CNN
        self.text_cnn = nn.Sequential(
            # Layer 1
            nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(self.drp),
            # Layer 2
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(self.drp),
            # Layer 3
            nn.Conv2d(3, 3, kernel_size=3, stride=(1, 10), padding=(1, 0)),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(self.drp),
        )

        # Image CNN
        # self.image_cnn = nn.Sequential(
        #     # Layer 1
        #     nn.Conv2d(3, 3, kernel_size=3, stride=3, padding=1),
        #     nn.BatchNorm2d(3),
        #     nn.ReLU(inplace=True),
        #     nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
        #     nn.Dropout(self.drp),
        #     # Layer 2
        #     nn.Conv2d(3, 3, kernel_size=3, stride=2, padding=1),
        #     nn.BatchNorm2d(3),
        #     nn.ReLU(inplace=True),
        #     nn.MaxPool2d(kernel_size=3, stride=1, padding=0),
        #     nn.Dropout(self.drp),
        #     # Layer 3
        #     nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=0),
        #     nn.BatchNorm2d(3),
        #     nn.ReLU(inplace=True),
        #     nn.MaxPool2d(kernel_size=5, stride=1, padding=0),
        #     nn.Dropout(self.drp),
        # )

        # Image + Text CNN
        self.combined_cnn = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 3, kernel_size=3, stride=3, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(self.drp),
        )

        # Labels linear
        self.labels_linear = nn.Sequential(
            nn.Linear(900, 450),
            nn.ReLU(inplace=True),
        )

        # All linear

        self.all_linear = nn.Sequential(
            nn.Linear(4350, 2000),
            nn.ReLU(),
            nn.Dropout(self.drp),
            nn.Linear(2000, 2000),
            nn.ReLU(),
            nn.Dropout(self.drp),
            nn.Linear(2000, 2000),
            nn.ReLU(),
            nn.Dropout(self.drp),
            nn.Linear(2000, 2000),
            nn.ReLU(),
            nn.Dropout(self.drp),
            nn.Linear(2000, 2000),
            nn.ReLU(),
            nn.Dropout(self.drp),
            nn.Linear(2000, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        img, text, labels = x

        # Process image
        cnn_img = model_conv(img)

        # Process text
        text_emb = self.embedding(text)
        lstm, _ = self.lstm(text_emb)
        lstm_single = lstm.reshape(lstm.shape[0], -1)
        lstm_out = self.lstm_linear(lstm_single)
        text_emb_cnn = torch.unsqueeze(text_emb, 1)
        cnn_text = self.text_cnn(text_emb_cnn)

        # Add text and image CNN
        # combined_cnn = cnn_text
        # combined_cnn = self.combined_cnn(combined_cnn)
        cnn_text = cnn_text.view(cnn_text.shape[0], -1)

        # Process labels
        labels_emb = self.embedding(labels)
        labels_single = labels_emb.view(labels.shape[0], 900)
        labels_out = self.labels_linear(labels_single)

        # Process all
        combined_all = torch.cat((lstm_out, labels_out, cnn_img, cnn_text), dim=1)
        # print(combined_all.shape)
        out = self.all_linear(combined_all)
        out = torch.squeeze(out)
        # print(out)
        return out

In [14]:
import time
import torch.nn.functional as F
from tqdm import tqdm

model = Model(text_obj.weights)
model.to(device)

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
epochs = 30

train_loss = []
val_loss = []

for epoch in range(epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.
    for (i, j, k) in zip(image_loader, text_loader, labels_loader):
        image = i[0].to(device)
        text = j[0].to(device)
        labels = k[0].to(device)
        output = model((image, text, labels)) # hacky way to do this?
        loss = loss_fn(output, i[1].type(torch.FloatTensor).to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(image_loader)
    
    model.eval()
    avg_val_loss = 0.
    accurate_pred = 0

    for (i, j, k) in zip(image_loader_test, text_loader_test, labels_loader_test):
        image = i[0].to(device)
        text = j[0].to(device)
        labels = k[0].to(device)
        output = model((image, text, labels)).detach()
        label = i[1].type(torch.FloatTensor).to(device)
        avg_val_loss += loss_fn(output, label).item() / len(image_loader_test)
        pred = np.where(output.cpu().numpy() > 0.5, 1, 0)
        correct_pred = (i[1].numpy() == pred)
        accurate_pred += sum(correct_pred)

    val_accuracy = accurate_pred / len(images_test)
    train_loss.append(avg_loss)
    val_loss.append(avg_val_loss)
    
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t val_acc={:.4f}  \t time={:.2f}s'.format(
                epoch + 1, epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))

Epoch 1/30 	 loss=35.4781 	 val_loss=49.0000  	 val_acc=0.5100  	 time=156.27s
Epoch 2/30 	 loss=35.5127 	 val_loss=49.0000  	 val_acc=0.5100  	 time=158.42s
Epoch 3/30 	 loss=35.5009 	 val_loss=49.0000  	 val_acc=0.5100  	 time=158.38s


KeyboardInterrupt: 

In [14]:
torch.save(model,'initial_model')

In [15]:
for (i, j, k) in zip(image_loader_test, text_loader_test, labels_loader_test):
    image = i[0].to(device)
    text = j[0].to(device)
    labels = k[0].to(device)
    output = model((image, text, labels)).detach()
    label = i[1].to(device)
    # avg_val_loss += loss_fn(output, label).item() / len(image_loader_test)
    pred = np.where(output.cpu().numpy() > 0.5, 1, 0)
    print(output)
    break

tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
